In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [26]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

In [90]:
# Create and fit forecaster
# ==============================================================================
rolling = RollingFeatures(stats= ['mean', 'median', 'sum'], window_sizes=[5, 4, 5])
forecaster = ForecasterAutoregDirect(
                 regressor       = LinearRegression(),
                 steps           = 2,
                 lags            = None,
                 window_features= rolling,
                 differentiation = None
             )

y = pd.Series(np.arange(10), name='y')
exog = pd.Series(np.arange(100, 110), name='exog')
exog_predict = pd.Series(np.arange(110, 112), name='exog')
exog_predict.index = pd.RangeIndex(start=10, stop=12, step=1)
forecaster.fit(y=y, exog=exog)

In [94]:
forecaster._create_predict_inputs(exog=exog_predict)

([array([[  7. ,   7.5,  35. , 110. ]]),
  array([[  7. ,   7.5,  35. , 111. ]])],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog'],
 [1, 2],
 Index([10, 11], dtype='int64'))

In [93]:
forecaster.create_predict_X(exog=exog_predict)

['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog']


,roll_mean_5,roll_median_4,roll_sum_5,exog
10,7.0,7.5,35.0,110.0
11,7.0,7.5,35.0,111.0


In [95]:
forecaster.predict(exog=exog_predict)

10    10.0
11    11.0
Name: pred, dtype: float64

In [35]:
rolling.transform_batch(y)

,roll_mean_4,roll_median_4,roll_sum_5
5,2.5,2.5,10.0
6,3.5,3.5,15.0
7,4.5,4.5,20.0
8,5.5,5.5,25.0
9,6.5,6.5,30.0


In [96]:
y = pd.Series(np.arange(5), name='y')
forecaster = ForecasterAutoregDirect(LinearRegression(), lags=3, steps=2)

forecaster.create_train_X_y(y=y)

(   lag_1  lag_2  lag_3
 4    2.0    1.0    0.0,
 {1: 3    3.0
  Name: y_step_1, dtype: float64,
  2: 4    4.0
  Name: y_step_2, dtype: float64})